# Lấy dữ liệu từ Soundcloud

## 1. Cài đặt thư viện và những hàm chung

### 1.1. Cài đặt các thư viện cần cho toàn bài

- json: để lưu dữ liệu từ API
- requests: dùng để gửi request kéo dữ liệu về
- selenium: dùng để chạy các file JS của một trang web
- bs4: để lấy các tag trong HTML của một trang web
- time: dùng để cho chương trình ngủ 1 khoảng thời gian trước khi tiếp tục
- pandas: dùng để ghi file csv

In [1]:
import json
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import pandas as pd
from selenium.webdriver.common.by import By

### 1.2. Các hàm chung

Hàm wait dùng để dừng lại cho tới khi lấy được thẻ tag cần lấy.
- driver: thường là chrome
- selector_name: ví dụ như là tên class hoặc tên của id

In [2]:
# wait
def wait(driver, selector_name):
    while (True):
        try:
            myElem = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CLASS_NAME, selector_name)))
            if myElem is not None:
                break
        except TimeoutException:
            break

Hàm scroll dùng để load hết page bằng cách kéo page xuống cuối cùng.
- driver: thường là chrome

In [3]:
# scroll
def scroll(driver):
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(2)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

Hàm printProgessBar dùng để xuất ra màn hình thanh load, để người dùng biết mình chạy được bao nhiêu rồi.

In [4]:
import sys

def printProgressBar(i,max,postText):
    n_bar =10 #size of progress bar
    j= i/max
    sys.stdout.write('\r')
    sys.stdout.write(f"[{'=' * int(n_bar * j):{n_bar}s}] {int(100 * j)}%  {postText}")
    sys.stdout.flush()

## 2. Phần lấy dữ liệu

### 2.1. Lấy danh sách các user

Link bên dưới dùng để lấy được client_id

In [5]:
my_url = "https://soundcloud.com/discover"

In [6]:
driver = webdriver.Chrome()
driver.get(my_url)
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
client_id = soup.find("div", {"id": "g_id_intermediate_iframe"}).get("data-src")
client_id = ((client_id.split('?')[1]).split('&')[0]).split('=')[1]

In [7]:
client_id

'gu6nGJ3NnV3h5RCD8Aj6XpDVaPsjbsiY'

set_link là biến dùng để lưu lại các link của user

In [8]:
set_link = []

Đoạn code dưới đây dùng để lấy các user được đề cập đến trong trang chủ của Soundcloud và sau đó vào từng link để lấy danh sách following của user này. Với cách này ta có thể lấy được rất nhiều user, nhưng trong phần này thì sẽ giới hạn số users được lấy là hơn 1000 người.

In [9]:
driver = webdriver.Chrome()
driver.get(my_url)
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
link = soup.find_all("a", {
    "class": "sc-link-secondary sc-link-light playableTile__usernameHeading audibleTile__usernameHeading sc-truncate sc-text-h4"})

for item in link:
    user_url = "https://soundcloud.com" + item.get("href")
    if user_url not in set_link:
        set_link.append(user_url)

for user_url in set_link:
    driver.get(user_url)
    wait(driver, 'profileHeaderInfo__userName')
    
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    user_name = soup.find("h2", {"class": "profileHeaderInfo__userName"})

    # Get followers
    follower_url = user_url + "/following"
    driver.get(follower_url)

    #Wait
    #scroll(driver)

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    followers = soup.find_all("a", {"class": "userBadgeListItem__heading"})
    for follower in followers:
        if follower not in set_link:
            set_link.append("https://soundcloud.com" + follower.get("href"))
    if len(set_link) > 1000:
            break

In [10]:
len(set_link)

1011

Hàm get_all_user_data ở dưới này dùng để lấy tất cả các user ID của danh sách link của các user. Ở đây, một câu hỏi đặt ra là tại sao không lấy lồng vào đoạn code phía trên mà lại tách ra. Câu trả lời đơn giản là vì đoạn code trên kia lấy các link của users được liệt kê ra trong một trang, chứ không vào từng trang nên không thể lấy từng user được. Đoạn code dưới đây có thể là đoạn code chạy lâu nhất chương trình. Qua phần API thì việc lấy dữ liệu cực kì nhanh, và đặc biệt là nhanh hơn parse bằng HTML rất rất nhiều

In [11]:
def get_all_user_data(set_link):
    #Get user id in SoundCloud/Discover by get meta tag in HEAD html
    set_data_user = []
    
    for i, link_user in enumerate(set_link):
        driver = webdriver.Chrome()        
        driver.get(link_user)
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        #Get user id & user name & user url
        id_user = soup.find("meta", property="twitter:app:url:googleplay")
        set_data_user.append(id_user["content"].strip("soundcloud://users:"))
        printProgressBar(i, len(set_link), "%")
        driver.close()
    
    return set_data_user

In [12]:
set_user = get_all_user_data(set_link)

[========= ] 99%  %

In [13]:
len(set_user)

1011

### 2.2. Lấy thông tin user qua API

#### 2.2.1. Lấy thông tin

Truyền vào danh sách các user ids và client id, hàm get_user_by_API sẽ lấy tất cả các thông tin của một user thông qua API bên dưới. Hàm này sẽ trả về danh sách thông tin của các users, bao gồm id, username, permalink_url, playlist_count, track_count, city, followers_count, followings_count, likes_count, avatar_url, created_at, kind.

In [14]:
def get_user_by_API(set_user, client_id):
    set_data_user = []
    # Select first column of the dataframe as a series
    
    for i, item in enumerate(set_user):
        one_data_user = []
        user_id = item
        api_url = f'https://api-v2.soundcloud.com/users/{user_id}?client_id={client_id}'
        # get request
        r = requests.get(api_url)
        # check whether it get the url
        # or if it got request, does it have a string "incomplete_results":true in the response it recieved?
        while r.status_code != requests.codes.ok or '"incomplete_results":true' in r:
            time.sleep(1)
            r = requests.get(api_url)
        #JSON processing
        user = json.loads(r.text)
        #Get one data user
        one_data_user.append(user['id'])
        one_data_user.append(user['username'])
        one_data_user.append(user['permalink_url'])
        one_data_user.append(user['playlist_count'])
        one_data_user.append(user['track_count'])
        one_data_user.append(user['city'])
        one_data_user.append(user['followers_count'])
        one_data_user.append(user['followings_count'])
        one_data_user.append(user['likes_count'])
        one_data_user.append(user['avatar_url'])
        one_data_user.append(user['created_at'])
        one_data_user.append(user['kind'])
        
        set_data_user.append(one_data_user)
        printProgressBar(i, len(set_user), "%")
        
    return set_data_user

Biến set_data_user sẽ lưu lại danh sách này để chuẩn bị cho việc lưu vào file csv

In [15]:
set_data_user = get_user_by_API(set_user, client_id)

[========= ] 99%  %

#### 2.2.2. Lưu thông tin của các users vào file csv

In [16]:
f_out = pd.DataFrame(set_data_user, columns=['User ID',
                                             'User Name',
                                             'User URL',
                                             'playlist_count',
                                             'track_count',
                                             'CITY',
                                             'followers_count',
                                             'followings_count',
                                             'likes_count',
                                             'avatar_url',
                                             'created_at',
                                             'kind'
                                            ])
f_out.to_csv('userAPI.csv', index=False, sep='\t', encoding='utf-8')

### 2.3. Lấy thông tin track qua API

Hàm createNoneObj dùng để tạo một list danh sách gồm n chữ None, hàm này dùng trong việc nếu người dùng không có track nào thì sẽ trả về một list 'None'

In [17]:
def createNoneObj(n):
    return ['None' for i in range(n)]

#### 2.3.1. Lấy thông tin

Hàm getTracksAPI sẽ lấy tất cả thông tin của các tracks của từng user. Tham số cần truyền vào là danh sách user ID, client id, và limit (được mặc định là 100, biến này dùng để xác định số bài hát lấy được của). Hàm sẽ trả về user_id, artwork_url, created_at, duration, genre, id, kind, last_modified, license, likes_count, permalink, permalink_url, public, reposts_count, sharing, tag_list, title, uri, display_date.

In [18]:
def getTracksAPI(set_user, client_id, limit=100):
    tracks = []
    temp1 = f'https://api-v2.soundcloud.com/users/'
    temp2 = f'/tracks?client_id={client_id}&limit={limit}'

    for i, user_id in enumerate(set_user):
        api = temp1 + user_id + temp2
        while True:
            r = requests.get(api)
            if r.status_code != 200:
                time.sleep(1)
            else:
                break
        y = json.loads(r.text)
        if len(y['collection']) == 0:
            tracks.append([user_id] + createNoneObj(18))
        else:
            for track in y['collection']:
                tracks.append([
                    track['user_id'],
                    track['artwork_url'],
                    track['created_at'],
                    track['duration'],
                    track['genre'],
                    track['id'],
                    track['kind'],
                    track['last_modified'],
                    track['license'],
                    track['likes_count'],
                    track['permalink'],
                    track['permalink_url'],
                    track['public'],
                    track['reposts_count'],
                    track['sharing'],
                    track['tag_list'],
                    track['title'],
                    track['uri'],
                    track['display_date'],
                ])
        printProgressBar(i, len(set_user), "%")
        
    return tracks

Biến set_tracks sẽ lưu lại danh sách này để chuẩn bị cho việc lưu vào file csv

In [19]:
set_tracks = getTracksAPI(set_user, client_id)

[========= ] 99%  %

#### 2.3.2. Lưu thông tin của các tracks vào file csv

In [20]:
f_out = pd.DataFrame(set_tracks, columns=['user_id',
                                             'artwork_url',
                                             'created_at',
                                             'duration',
                                             'genre',
                                             'id',
                                             'kind',
                                             'last_modified',
                                             'license',
                                             'likes_count',
                                             'permalink',
                                             'permalink_url',
                                             'public',
                                             'reposts_count',
                                             'sharing',
                                             'tag_list',
                                             'title',
                                             'uri',
                                             'display_date'
                                            ])
f_out.to_csv('trackAPI.csv', index=False, sep='\t', encoding='utf-8')

### 2.4. Lấy thông tin playlists của từng user qua API

#### 2.4.1. Lấy thông tin playlists

Hàm getPlaylistsAPI sẽ lấy tất cả thông tin của các playlists của từng user. Tham số cần truyền vào là danh sách user ID, client id, và limit (được mặc định là 100, biến này dùng để xác định số bài hát lấy được của). Hàm sẽ trả về user_id, artwork_url, created_at, duration, embeddable_by, genre, id, kind, label_name, last_modified, license, likes_count, managed_by_feeds, permalink, permalink_url, public, reposts_count, sharing, tag_list, title, uri, is_album, published_at, display_date, allTrackIDs.

Trong đó allTrackIDs sẽ được ghép lại từ các trackID thuộc playlist đó. Mọi thông tin này đều được API trả về, nên chỉ cần cắt ghép các track ID.

In [21]:
def getPlaylistsAPI(set_user, client_id, limit=100):
    playlists = []
    
    for i, user_id in enumerate(set_user):
        
        url = f'https://api-v2.soundcloud.com/users/{user_id}/playlists?client_id={client_id}&limit={limit}'
        r = requests.get(url)

        while r.status_code != requests.codes.ok or '"incomplete_results":true' in r:
            time.sleep(1)
            r = requests.get(url)

        y = json.loads(r.text)
        if len(y['collection']) == 0:
            playlists.append([user_id]+createNoneObj(24))
        else:
            for playlist in y['collection']:
                trackIDs = []
                for track in playlist['tracks']:
                    trackIDs.append(str(track['id']))
                allTrackIDs = ','.join(trackIDs)
                playlists.append([playlist['user_id'],
                                 playlist['artwork_url'],
                                 playlist['created_at'],
                                 playlist['duration'],
                                 playlist['embeddable_by'],
                                 playlist['genre'],
                                 playlist['id'],
                                 playlist['kind'],
                                 playlist['label_name'],
                                 playlist['last_modified'],
                                 playlist['license'],
                                 playlist['likes_count'],
                                 playlist['managed_by_feeds'],
                                 playlist['permalink'],
                                 playlist['permalink_url'],
                                 playlist['public'],
                                 playlist['reposts_count'],
                                 playlist['sharing'],
                                 playlist['tag_list'],
                                 playlist['title'],
                                 playlist['uri'],
                                 playlist['is_album'],
                                 playlist['published_at'],
                                 playlist['display_date'],
                                 allTrackIDs])
        printProgressBar(i, len(set_user), "%")

    return playlists

Biến get_playlist sẽ lưu lại danh sách này để chuẩn bị cho việc lưu vào file csv

In [22]:
get_playlist = getPlaylistsAPI(set_user, client_id)

[========= ] 99%  %

#### 2.4.2. Lưu thông tin của các playlists vào file csv

In [23]:
f_out = pd.DataFrame(get_playlist, columns=['user_id',
                                 'artwork_url',
                                 'created_at',
                                 'duration',
                                 'embeddable_by',
                                 'genre',
                                 'id',
                                 'kind',
                                 'label_name',
                                 'last_modified',
                                 'license',
                                 'likes_count',
                                 'managed_by_feeds',
                                 'permalink',
                                 'permalink_url',
                                 'public',
                                 'reposts_count',
                                 'sharing',
                                 'tag_list',
                                 'title',
                                 'uri',
                                 'is_album',
                                 'published_at',
                                 'display_date',
                                 'allTrackIDs'])
f_out.to_csv('playlistAPI.csv', index=False, sep='\t', encoding='utf-8')